In [57]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from bertopic.representation import OpenAI
import openai
import tiktoken
import os
from hdbscan import HDBSCAN
import re

In [24]:
# 1) Load and prepare text
df = pd.read_csv("../data/interactions.csv")

# combine input/output into a single text field
df["combined"] = (df["input"].fillna("").astype(str) + " " + df["output"].fillna("").astype(str)).str.strip()

# choose docs = one row per interaction (matches probs shape to df rows)
docs = df["combined"].tolist()

hdbscan_model = HDBSCAN(min_cluster_size=25, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

prompt = """
Your job is to generate a topic label that represents a type of user query or task for LLMs. 

Determine the topic label based on the following query-response pairs: \n[DOCUMENTS]

The topic is described by the following keywords: [KEYWORDS]

Based on the above information, can you give a short label of the topic? Only output your short label with no other text or annotations.
"""

# Tokenizer
tokenizer= tiktoken.encoding_for_model("gpt-4o")

# Create your representation model
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
representation_model = OpenAI(
    client,
    model="gpt-4o",
    delay_in_seconds=2,
    chat=True,
    nr_docs=5,
    doc_length=100,
    tokenizer=tokenizer
)

# 2) Fit BERTopic
umap_model = UMAP(random_state=11)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2)
topic_model = BERTopic(
    umap_model=umap_model,
    vectorizer_model=vectorizer_model,
    hdbscan_model=hdbscan_model,
    representation_model=representation_model,
    calculate_probabilities=True,
    top_n_words=10,
    verbose=True
)

topics_assigned, probs = topic_model.fit_transform(docs)

# 3) Save topic summary
topic_info = topic_model.get_topic_info()  # includes -1 "outliers" row first
#topic_info.to_csv("../data/topics.csv", index=False)

# 4) Build a labeled probability DataFrame (columns = topic names, excluding -1)
# The probability matrix `probs` has shape (n_docs, n_topics_without_outliers)
# Align names by taking non -1 topics from topic_info in order.
non_outlier = topic_info[topic_info["Topic"] != -1].copy()

# Ensure we have exactly as many names as there are columns in probs
expected_cols = probs.shape[1]
names = non_outlier["Name"].tolist()[:expected_cols]
ids = non_outlier["Topic"].tolist()[:expected_cols]

probs_df = pd.DataFrame(probs, columns=names)

# 5) Attach assigned topic and probs to the original rows
df_out = df.reset_index(drop=True).copy()
df_out["assigned_topic"] = topics_assigned  # the single best topic per row
df_out = pd.concat([df_out, probs_df.reset_index(drop=True)], axis=1)

# 6) Save
#df_out.to_csv("../data/interactions_with_topic_probs.csv", index=False)

2025-09-07 15:01:31,361 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/108 [00:00<?, ?it/s]

2025-09-07 15:01:42,584 - BERTopic - Embedding - Completed ✓
2025-09-07 15:01:42,585 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-09-07 15:01:50,037 - BERTopic - Dimensionality - Completed ✓
2025-09-07 15:01:50,038 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-09-07 15:01:50,220 - BERTopic - Cluster - Completed ✓
2025-09-07 15:01:50,224 - BERTopic - Representation - Fine-tuning topics using representation models.
100%|███████████████████████████████████████████| 35/35 [01:29<00:00,  2.57s/it]
2025-09-07 15:03:20,655 - BERTopic - Representation - Completed ✓


In [26]:
topic_info.to_csv("../data/topics_reduced.csv", index=False)
df_out.to_csv("../data/interactions_with_topic_probs_reduced.csv", index=False)

In [77]:
df = pd.read_csv("../data/regression/topic_coefficients.csv")
df = df[df["p_value"]<0.10]
df = df[((df["coefficient"]<=-0.04)&(df["y_var"]=="mimesis"))|((df["coefficient"]<=-0.005)&(df["y_var"]=="sycophancy"))]
df = df[df["topic_column"]!="X12_Response.Error.Message"]
interactions = pd.read_csv("../data/interactions_with_topics.csv")
topics_by_user = interactions.groupby(["assigned_topic"])["user_id"].nunique().reset_index()
TOPICS_MULTIPLE_USERS = topics_by_user.loc[topics_by_user["user_id"]>=np.median(topics_by_user["user_id"]), "assigned_topic"].to_list()

In [78]:
for y_var in df["y_var"].unique():
    curr_out = df[df["y_var"]==y_var]
    for model_name in curr_out["model_name"].unique():
        for task_name in curr_out["task_name"].unique():
            print(y_var, model_name, task_name)
            curr = curr_out[curr_out["model_name"]==model_name]
            curr = curr[curr["task_name"]==task_name]
            curr["topic_number"] = curr["topic_column"].str.extract(r'(\d+)')
            curr["topic_number"] = curr["topic_number"].astype(int)
            curr = curr[curr["topic_number"].isin(TOPICS_MULTIPLE_USERS)]
            print(len(curr))
            #curr["topic_column"] = curr["topic_column"].str.replace(r"^X\d+_", "", regex=True)
            for i,r in curr.sort_values(by='coefficient', ascending=False).iterrows():
                t = r["topic_column"]
                t = re.sub(r"^X\d+_", "", t)
                t = re.sub(r"\.", " ", t)
                coeff = round(r["coefficient"], 2)
                coeff_str = f"{coeff:.2f}"
                print(t, "("+coeff_str+")", end=", ")
            print()
            print()

sycophancy claude-sonnet-4-20250514 aita
1
Polite Communication Techniques (-0.03), 

mimesis claude-sonnet-4-20250514 aita
6
2025 Release Dates (-0.04), Honda Starter Troubleshooting (-0.04), Cloud Storage Management (-0.05), VS Code Navigation (-0.08), Career Development Strategies (-0.08), Travel and Safety (-0.09), 

mimesis claude-sonnet-4-20250514 politics
4
Mechanical Engineering Robotics (-0.08), Cloud Storage Management (-0.08), VS Code Navigation (-0.09), Thermodynamics Applications (-0.15), 

mimesis gpt-4.1-mini-2025-04-14 aita
6
Cloud Storage Management (-0.05), Exam and Assignment Structure (-0.07), 2025 Release Dates (-0.09), Classical Latin and Music (-0.10), VS Code Navigation (-0.12), Travel and Safety (-0.14), 

mimesis gpt-4.1-mini-2025-04-14 politics
1
Polite Communication Techniques (-0.17), 



In [33]:
        
interactions = pd.read_csv("../data/interactions_with_topic_probs.csv")
interactions = interactions.groupby(["assigned_topic", "user_id"])["timestamp"].count().reset_index()
interactions = interactions.rename(columns={"timestamp": "num_queries"})
interactions = interactions[interactions["assigned_topic"]>=0]
participants = pd.read_csv("../data/participants.csv")
participants = participants[["user_id", "political_lean", "gender"]]
participants.loc[participants["political_lean"].isin(["Moderate", "Conservative", "Very Conservative"]), "political_lean"] = "right"
participants.loc[participants["political_lean"].isin(["Liberal", "Very Liberal"]), "political_lean"] = "left"

interactions = interactions.merge(participants, on="user_id", how="left")
topics = pd.read_csv("../data/topics.csv")
topics["assigned_topic"] = topics["Topic"]
topics["topic_name"] = topics["Name"]
topics = topics[["assigned_topic", "topic_name"]]

interactions = interactions.merge(topics, on="assigned_topic", how="left")
interactions = interactions.drop(columns=["assigned_topic"])

interactions = interactions.groupby(["political_lean", "topic_name"])["user_id"].count().reset_index()
participants = pd.read_csv("../data/participants.csv")
participants = participants[participants["passed_attention"]=="yes"]
participants = participants[["user_id", "political_lean", "gender"]]
participants.loc[participants["political_lean"].isin(["Moderate", "Conservative", "Very Conservative"]), "political_lean"] = "right"
participants.loc[participants["political_lean"].isin(["Liberal", "Very Liberal"]), "political_lean"] = "left"
participants["total"] = participants["user_id"]
participants = participants.groupby(["political_lean"])["total"].count().reset_index()
interactions = interactions.merge(participants, on="political_lean", how="left")
interactions["prop_users"] = interactions["user_id"]/interactions["total"]
topics_all = interactions.groupby(["topic_name"])["user_id"].sum().reset_index()
topics_all.sort_values(by='user_id', ascending=False).head(10)

,topic_name,user_id
1,10_today_hello_assist_hey,24
11,1_welcome_youre_im_day,22
0,0_email_thank_casual_formal,20
22,2_water_clean_dry_coffee,16
44,4_credit_card_income_bank,16
3,12_occured_generating_sorry_error,13
20,28_conversation_topics_talk_dog,13
55,5_clients_emotions_profile_therapy,12
9,18_beans_rice_sauce_pie,12
27,34_points_total_attendance_grade,10


In [216]:
interactions = pd.read_csv("../data/interactions_with_topic_probs.csv")
interactions = interactions.groupby(["assigned_topic", "user_id"])["timestamp"].count().reset_index()
interactions = interactions.rename({"timestamp": "num_queries"})
interactions = interactions[interactions["assigned_topic"]>=0]
participants = pd.read_csv("../data/participants.csv")
participants = participants[["user_id", "political_lean", "gender"]]
participants.loc[participants["political_lean"].isin(["Moderate", "Conservative", "Very Conservative"]), "political_lean"] = "right"
participants.loc[participants["political_lean"].isin(["Liberal", "Very Liberal"]), "political_lean"] = "left"

topics_by_demographic = interactions.merge(participants, on="user_id", how="left")

for model_name in df["model_name"].unique():
    for task_name in df["task_name"].unique():
        print(model_name, task_name)
        curr = df[df["model_name"]==model_name]
        curr = curr[curr["task_name"]==task_name]
        curr["topic_number"] = curr["topic_column"].str.extract(r'(\d+)')
        curr["topic_number"] = curr["topic_number"].astype(int)
        curr = curr[curr["topic_number"].isin(topics_by_user["assigned_topic"])]
        curr = topics_by_demographic[topics_by_demographic["assigned_topic"].isin(curr["topic_number"])].groupby(["political_lean", "gender"])["user_id"].nunique().reset_index()
        print(curr)
        print()

SyntaxError: unmatched ']' (2182744335.py, line 20)

In [244]:
understanding = pd.read_csv("../data/survey_results.csv")
understanding = understanding[understanding["understanding"]==5]
understanding[["participant", "understanding", "model", "task"]].drop_duplicates()

interactions = pd.read_csv("../data/interactions_with_topic_probs.csv")
interactions = interactions.groupby(["assigned_topic", "user_id"])["timestamp"].count().reset_index()
interactions = interactions.rename({"timestamp": "num_queries"})

for model_name in understanding["model"].unique():
    for task_name in understanding["task"].unique():
        print(model_name, task_name)
        curr = understanding[(understanding["model"]==model_name)&(understanding["task"]==task_name)]
        topics = interactions[interactions["user_id"].isin(curr["participant"])]
        topics = topics[topics["assigned_topic"]>=0]
        print(topics["assigned_topic"].value_counts().head(10).reset_index()["assigned_topic"].to_list())
        print()

claude-sonnet-4-20250514 aita
[1, 10, 0, 12, 28, 2, 4, 5, 7, 47]

claude-sonnet-4-20250514 politics
[1, 10, 0, 4, 2, 12, 27, 47, 34, 18]

gpt-4.1-mini-2025-04-14 aita
[1, 10, 0, 12, 2, 28, 4, 5, 34, 7]

gpt-4.1-mini-2025-04-14 politics
[1, 0, 10, 12, 4, 2, 5, 47, 27, 34]



[1, 0, 10, 12, 4, 2, 5, 47, 27, 34]

In [206]:
topics_by_demographic[topics_by_demographic["assigned_topic"].isin(curr["topic_number"])].groupby(["political_lean", "gender"])["user_id"].nunique().reset_index()

,political_lean,gender,user_id
0,left,man,2
1,left,woman,3
2,right,man,4
3,right,non-binary,1
4,right,woman,2


,political_lean,gender,user_id
0,left,man,3
1,left,woman,5
2,right,man,9
3,right,non-binary,1
4,right,woman,4
